In [1]:
import pandas as pd
import numpy as np
import lib.Model as md

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
raw_train_data = pd.read_csv('./input/train.csv')
raw_test_data = pd.read_csv('./input/test.csv')

weather_data = pd.read_csv('./input/weather.csv')
spray_data = pd.read_csv('./input/spray.csv')

In [3]:
def dummy_species(data): # TODO 高速化
    # 蚊の種類をone-hot形式にする
    sp_keys = {'CULEX ERRATICUS' , 'CULEX PIPIENS', 'CULEX RESTUANS',
 'CULEX SALINARIUS', 'CULEX TARSALIS', 'CULEX TERRITANS' , 'CULEX PIPIENS/RESTUANS'}
    sp_map = {'CULEX ERRATICUS' : np.array([1, 0, 0, 0, 0, 0]),
                        'CULEX PIPIENS' : np.array([0, 1, 0, 0 , 0, 0]),
                        'CULEX RESTUANS' : np.array([0, 0, 1, 0, 0, 0]),
                        'CULEX SALINARIUS' : np.array([0, 0, 0, 1, 0, 0]),
                        'CULEX TARSALIS' : np.array([0, 0, 0, 0, 1, 0]),
                        'CULEX TERRITANS' : np.array([0, 0, 0, 0, 0, 1]),
                        'CULEX PIPIENS/RESTUANS' : np.array([0, 1, 1, 0, 0, 0])}
    dummies = np.empty((0, len(sp_keys)-1), int)
    for i in data:
        if i not in sp_keys:
            dummies = np.append(dummies, [np.zeros(len(sp_keys)-1)], axis=0)
        else:
            dummies = np.append(dummies, [np.array(sp_map[i])], axis=0)
    return pd.DataFrame(dummies)

def dummy_trap(data): # TODO 高速化
    # trapの種類をone-hot形式にする
    trap_keys = list(set(raw_train_data['Trap']))
    trap_dummy = pd.get_dummies(trap_keys)
    dummies = np.empty((0, len(trap_keys)), int)
    for i in data:
            if i not in trap_keys:
                dummies = np.append(dummies, [np.zeros(len(trap_keys))], axis=0)
            else:
                dummies = np.append(dummies, [np.array(trap_dummy[i])], axis=0)
    return pd.DataFrame(dummies)
                
def preprocessing(data, weather, spray):
    # pandasのデータで前処理をしてnumpy配列を返す
    
    # 天気情報の前処理
    formated_weather = weather.drop_duplicates(subset='Date').drop(['Station', 'CodeSum', 'Depth', 'Water1'], axis=1)
    formated_weather = formated_weather.convert_objects(convert_numeric=True).fillna(0) # TODO 欠損値の扱い

    # TODO スプレーデータの前処理
    
    # メインデータの前処理
    #if 'NumMosquitos' in data: # TODO 重複データの扱い
    #    data = data.drop_duplicates(subset=['Date', 'Species', 'Latitude', 'Longitude', 'Trap'])
    
    species = dummy_species(data['Species'])
    trap = dummy_trap(data['Trap'])

    merged_data = pd.merge(data, formated_weather, on='Date')
    merged_data = pd.merge(merged_data, species, right_index=True, left_index=True)
    merged_data = pd.merge(merged_data, trap, right_index=True, left_index=True)

    if 'NumMosquitos' in merged_data:
        preprocessed_data = merged_data.drop(['Date', 'Address', 'Species', 'Block', 'Street', 'Trap', 'AddressNumberAndStreet', 'AddressAccuracy', 'NumMosquitos', 'WnvPresent'], axis=1)
        return np.array(preprocessed_data), np.array(data['NumMosquitos']), np.array(data['WnvPresent'])
    else:
        preprocessed_data = merged_data.drop(['Id','Date', 'Address', 'Species', 'Block', 'Street', 'Trap', 'AddressNumberAndStreet', 'AddressAccuracy'], axis=1)
        return np.array(preprocessed_data), np.array([]), np.array([])

In [4]:
print("Make training data")
train_data, num_mosquitos, label = preprocessing(raw_train_data, weather_data, spray_data)
print("Make test data")
test_data, tmp1, tmp2 = preprocessing(raw_test_data, weather_data, spray_data)

Make training data


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:37: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


Make test data


In [5]:
# SVM
model_svm = md.SVM(optimization=True)
model_svm.fit(train_data, label)
result_svm = model_svm.predict(test_data)

In [6]:
# RandomForest
model_rf = md.RandomForest()
model_rf.fit(train_data, label)
result_rf = model_rf.predict(test_data)

In [7]:
# LightGBM
model_lgbm = md.LightGBM()
model_lgbm.fit(train_data, label)
result_lgbm = model_lgbm.predict(test_data)

In [8]:
# GLM
model_glm = md.GLM()
model_glm.fit(train_data, label, num_mosquitos)
result_glm = model_glm.predict(test_data)

100 [8.69804861e-04 3.79810409e-04 2.96732852e-04 2.25169912e-04
 5.79998729e-04 2.62918354e-04 6.96726260e-04 7.42422364e-04
 1.54368044e-04 4.20348089e-04 4.94559900e-04 9.53064771e-04
 7.55199154e-04 4.72305435e-05 6.10336244e-04 5.32939169e-05
 7.16570281e-04 7.88743376e-04 2.26265681e-04 6.27760640e-04
 1.04213186e-05 7.85706044e-04 1.38041753e-04 8.36004941e-04
 5.82926149e-04 7.45367692e-04 7.03991736e-04 5.54845465e-04
 2.94430871e-04 1.30920607e-04 7.05337722e-04 4.57532327e-04
 3.73103466e-04 9.80710544e-04 5.89478663e-04 2.07380104e-04
 3.92130683e-04 7.29717241e-04 6.72212538e-04 2.40540926e-04
 4.73629809e-04 1.76057418e-04 9.35624109e-04 2.36378891e-04
 9.77438533e-04 2.56210869e-04 5.91906144e-04 9.02081488e-04
 4.20257569e-04 5.53225534e-04 5.19578286e-04 1.72798404e-04
 9.85713246e-04 5.04256463e-04 2.49020555e-04 7.22647137e-04
 3.87981514e-04 1.60765949e-04 4.44030796e-04 4.90332999e-04
 6.61774324e-04 5.44014097e-04 5.04531549e-04 9.55212003e-04
 8.69064630e-04 5.87

500 [8.69804861e-04 3.79810409e-04 2.96732852e-04 2.25169912e-04
 5.79998729e-04 2.62918354e-04 6.96726260e-04 7.42422364e-04
 1.54368044e-04 4.20348089e-04 4.94559900e-04 9.53064771e-04
 7.55199154e-04 4.72305435e-05 6.10336244e-04 5.32939169e-05
 7.16570281e-04 7.88743376e-04 2.26265681e-04 6.27760640e-04
 1.04213186e-05 7.85706044e-04 1.38041753e-04 8.36004941e-04
 5.82926149e-04 7.45367692e-04 7.03991736e-04 5.54845465e-04
 2.94430871e-04 1.30920607e-04 7.05337722e-04 4.57532327e-04
 3.73103466e-04 9.80710544e-04 5.89478663e-04 2.07380104e-04
 3.92130683e-04 7.29717241e-04 6.72212538e-04 2.40540926e-04
 4.73629809e-04 1.76057418e-04 9.35624109e-04 2.36378891e-04
 9.77438533e-04 2.56210869e-04 5.91906144e-04 9.02081488e-04
 4.20257569e-04 5.53225534e-04 5.19578286e-04 1.72798404e-04
 9.85713246e-04 5.04256463e-04 2.49020555e-04 7.22647137e-04
 3.87981514e-04 1.60765949e-04 4.44030796e-04 4.90332999e-04
 6.61774324e-04 5.44014097e-04 5.04531549e-04 9.55212003e-04
 8.69064630e-04 5.87

900 [8.76300158e-04 3.75806812e-04 2.98975139e-04 2.26543014e-04
 5.84814078e-04 2.64197818e-04 7.02423925e-04 7.49089283e-04
 1.55226501e-04 4.22426441e-04 5.06271745e-04 9.61504792e-04
 7.49487513e-04 4.74300605e-05 6.14404203e-04 5.35505370e-05
 7.19914353e-04 7.93687752e-04 2.27293005e-04 6.24507730e-04
 1.04682216e-05 7.89937589e-04 1.37565255e-04 8.32227469e-04
 5.79263660e-04 7.43401157e-04 7.05381211e-04 5.54857963e-04
 2.93584120e-04 1.30271887e-04 7.02839612e-04 4.56356478e-04
 3.71747739e-04 9.80342161e-04 5.88080271e-04 2.06602798e-04
 3.92094506e-04 7.28937284e-04 6.70797569e-04 2.40259008e-04
 4.73247124e-04 1.75616768e-04 9.26870265e-04 2.35099280e-04
 9.81706574e-04 2.53360951e-04 5.94015958e-04 9.01305159e-04
 4.19396083e-04 5.52295803e-04 5.19173114e-04 1.70712722e-04
 9.88171617e-04 5.02844234e-04 2.47894239e-04 7.22415990e-04
 3.88637626e-04 1.60470802e-04 4.43396616e-04 4.89138064e-04
 6.62034418e-04 5.43749757e-04 5.03919288e-04 9.53703862e-04
 8.73626096e-04 5.86

1300 [8.77141551e-04 3.75712309e-04 2.98129589e-04 2.23677877e-04
 5.86181242e-04 2.61302445e-04 7.04088744e-04 7.50321753e-04
 1.54806973e-04 4.19264602e-04 5.06773061e-04 9.95022756e-04
 7.51483939e-04 4.73191602e-05 6.69389695e-04 5.30687213e-05
 7.15247594e-04 7.80539948e-04 2.29342889e-04 6.28394225e-04
 1.04423712e-05 7.87820072e-04 1.37032210e-04 8.34805476e-04
 5.82896492e-04 7.40932603e-04 7.04784609e-04 5.55254044e-04
 2.92462677e-04 1.30852421e-04 7.00165791e-04 4.54894735e-04
 3.70369157e-04 9.81845006e-04 5.86414100e-04 2.07425092e-04
 3.92489293e-04 7.28854856e-04 6.69336841e-04 2.40207507e-04
 4.73283027e-04 1.75075486e-04 9.35315666e-04 2.37043681e-04
 9.79342453e-04 2.56089277e-04 5.92850916e-04 9.01345832e-04
 4.18535333e-04 5.51608346e-04 5.19893842e-04 1.72703596e-04
 9.86953606e-04 5.00956777e-04 2.47711508e-04 7.23441665e-04
 3.88393052e-04 1.60218291e-04 4.43056313e-04 4.87690866e-04
 6.62304375e-04 5.46066193e-04 5.04805380e-04 9.52709500e-04
 8.71277514e-04 5.8

1700 [8.78569621e-04 3.85007788e-04 2.99069209e-04 2.26487863e-04
 5.85694677e-04 2.65067274e-04 7.03502669e-04 7.50282406e-04
 1.55199684e-04 4.23149642e-04 5.11997883e-04 1.03411051e-03
 7.48701139e-04 4.74260228e-05 6.21516196e-04 5.36953503e-05
 7.18988662e-04 7.93704965e-04 2.31719333e-04 6.21753676e-04
 1.04680215e-05 7.91768565e-04 1.37220434e-04 8.31498922e-04
 5.80007072e-04 7.42311493e-04 7.05097216e-04 5.54649122e-04
 2.93071228e-04 1.29786769e-04 7.00970166e-04 4.55700478e-04
 3.70697044e-04 9.78535089e-04 5.87269375e-04 2.06675956e-04
 3.90682990e-04 7.35190699e-04 6.69919930e-04 2.43317636e-04
 4.74995199e-04 1.75368483e-04 9.29490568e-04 2.33900210e-04
 9.81655663e-04 2.54350555e-04 5.93885841e-04 9.05928115e-04
 4.18844335e-04 5.51521520e-04 5.19021454e-04 1.71487465e-04
 9.87812090e-04 5.02001593e-04 2.46573371e-04 7.20727204e-04
 3.88476219e-04 1.60252698e-04 4.42740418e-04 4.88456022e-04
 6.61768738e-04 5.52283669e-04 5.03803472e-04 9.52337386e-04
 8.73787772e-04 5.9

...

8100 [ 8.74909682e-04  7.20282365e-04  2.75149623e-04  2.21966063e-04
  5.22091027e-04  2.67915163e-04  6.68886281e-04  7.43743845e-04
  1.34781625e-04  4.06046861e-04  9.17726232e-04  3.40027986e-03
  6.98644886e-04  5.36601692e-05  1.05569550e-03  5.44953573e-05
  5.28569409e-04  9.91758575e-04  4.59197168e-04  6.07419322e-04
  1.10207595e-05  9.52176204e-04  1.18567854e-04  9.66518152e-04
  7.57632968e-04  8.08214384e-04  7.07676687e-04  7.74463161e-04
  3.32299574e-04  1.18859894e-04  6.24920933e-04  4.45569438e-04
  5.00597736e-04  1.08431561e-03  6.31921566e-04  1.60782612e-04
  4.28029675e-04  7.24049236e-04  4.84931617e-04  2.09290809e-04
  7.05767708e-04  1.94377221e-04  7.94726220e-04  2.31549326e-04
  1.06801591e-03  1.64138036e-04  3.26434622e-04  1.23630944e-03
  3.32119172e-04  5.76466470e-04  6.30775911e-04  1.18217816e-04
  1.02863623e-03  5.58962618e-04  1.89989001e-04  7.75112225e-04
  3.95339189e-04  1.92200505e-04  4.43372072e-04  5.22263861e-04
  6.66580120e-04  1.

8500 [ 8.63184974e-04  7.71199692e-04  2.74287751e-04  2.22914902e-04
  5.24889484e-04  2.67645214e-04  6.68728459e-04  7.52507151e-04
  1.27872731e-04  4.01278361e-04  9.84131296e-04  3.77830908e-03
  7.24138345e-04  5.43627080e-05  1.11670917e-03  5.41704251e-05
  5.05462814e-04  1.01211780e-03  4.92355686e-04  6.08367582e-04
  1.10547769e-05  9.93846184e-04  1.15479617e-04  9.73563348e-04
  7.96099109e-04  8.14891809e-04  7.13114682e-04  8.03596544e-04
  3.37900357e-04  1.16302870e-04  6.13858871e-04  4.36090128e-04
  5.21625721e-04  1.09142743e-03  6.38291924e-04  1.54100710e-04
  4.23541880e-04  7.31676915e-04  4.53776147e-04  1.99155673e-04
  6.93113566e-04  1.96928520e-04  7.97125237e-04  2.32473692e-04
  1.07601808e-03  1.48152154e-04  2.81505921e-04  1.28791071e-03
  3.18372952e-04  5.80371312e-04  6.61026202e-04  1.12222052e-04
  1.01923125e-03  5.66587694e-04  1.80674370e-04  7.71701100e-04
  4.00302583e-04  2.00309099e-04  4.46871339e-04  5.24938510e-04
  6.82071222e-04  1.

8900 [ 8.53870021e-04  8.34123034e-04  2.71659435e-04  2.31160989e-04
  5.11150839e-04  2.75587536e-04  6.66233997e-04  7.42194110e-04
  1.27599774e-04  3.86896338e-04  1.06630030e-03  4.20881373e-03
  7.04731710e-04  5.50536866e-05  1.17159007e-03  5.42107626e-05
  4.64684269e-04  1.02276894e-03  5.36083519e-04  6.01898067e-04
  1.11186328e-05  1.03161031e-03  1.11694323e-04  9.98691434e-04
  8.30307104e-04  8.31566736e-04  7.15010670e-04  8.35991407e-04
  3.47005408e-04  1.15844327e-04  5.96890162e-04  4.45742165e-04
  5.41598989e-04  1.10806616e-03  6.51077015e-04  1.43754538e-04
  4.30567680e-04  7.45185524e-04  4.19635722e-04  1.92932221e-04
  7.41284301e-04  1.98834109e-04  7.90400695e-04  2.28615987e-04
  1.11874249e-03  1.31100128e-04  2.30728507e-04  1.35825372e-03
  3.04222958e-04  5.86432271e-04  6.88544059e-04  1.03285721e-04
  1.02306838e-03  5.86031888e-04  1.70946196e-04  7.79324807e-04
  4.02854150e-04  2.18306965e-04  4.49065134e-04  5.28891723e-04
  6.86193316e-04  1.

9300 [ 8.53486233e-04  8.95206665e-04  2.65144171e-04  2.35219033e-04
  5.07831441e-04  2.82021024e-04  6.62880757e-04  7.30182923e-04
  1.21226649e-04  3.80420823e-04  1.13603372e-03  4.70537027e-03
  7.45411403e-04  5.57744258e-05  1.23550894e-03  5.38428264e-05
  4.32990713e-04  1.06154848e-03  5.95518703e-04  5.98017569e-04
  1.11933297e-05  1.10411819e-03  1.07336602e-04  1.00536517e-03
  8.70126474e-04  8.51733212e-04  7.20623651e-04  8.78271354e-04
  3.56812356e-04  1.11561139e-04  5.79327090e-04  3.60453820e-04
  5.62832042e-04  1.11477828e-03  7.02269655e-04  1.36749478e-04
  4.29484352e-04  7.54204516e-04  3.39597675e-04  1.86267194e-04
  7.91939704e-04  2.28355958e-04  7.87798373e-04  2.23564972e-04
  1.09838004e-03  1.12685937e-04  1.77445675e-04  1.43088130e-03
  2.68054254e-04  5.47330466e-04  7.30845693e-04  9.47593922e-05
  1.01729547e-03  6.01801221e-04  1.56084950e-04  7.75931259e-04
  4.08478215e-04  2.18515427e-04  4.57885001e-04  5.85834775e-04
  7.05960608e-04  1.

9700 [ 8.44164159e-04  9.74733303e-04  2.62797870e-04  2.42603730e-04
  5.00836439e-04  2.89323633e-04  6.38734291e-04  7.06476351e-04
  1.15800112e-04  3.64355830e-04  1.25418739e-03  5.26767147e-03
  7.45917664e-04  5.54518357e-05  1.32112606e-03  5.36204424e-05
  3.85634915e-04  1.09922596e-03  6.53688625e-04  5.93032113e-04
  1.11049262e-05  1.15178503e-03  1.01473761e-04  1.03194912e-03
  9.21988666e-04  8.68238337e-04  7.24599305e-04  9.05449977e-04
  3.73483897e-04  1.10160071e-04  5.50883154e-04  4.39299919e-04
  5.90885927e-04  1.14891593e-03  6.85526048e-04  1.24025847e-04
  4.37357681e-04  7.72692816e-04  3.16236327e-04  1.73379904e-04
  8.57453121e-04  2.08981908e-04  7.78820757e-04  2.20618977e-04
  1.09916198e-03  9.07132684e-05  9.34452819e-05  1.52343675e-03
  2.64951182e-04  5.65745588e-04  7.70768714e-04  8.34356925e-05
  1.01342063e-03  6.26264943e-04  1.40295256e-04  7.91059010e-04
  4.12725473e-04  2.36974717e-04  4.61289762e-04  5.50810182e-04
  7.14021477e-04  1.

In [9]:
submitData_svm = pd.DataFrame.from_dict({"Id": raw_test_data['Id'], "WnvPresent": result_svm})
submitData_rf = pd.DataFrame.from_dict({"Id": raw_test_data['Id'], "WnvPresent": result_rf})
submitData_lgbm = pd.DataFrame.from_dict({"Id": raw_test_data['Id'], "WnvPresent": result_lgbm})
submitData_glm = pd.DataFrame.from_dict({"Id": raw_test_data['Id'], "WnvPresent": result_glm})

In [10]:
submitData_svm.to_csv('submission_svm.csv', index=0)
submitData_rf.to_csv('submission_rf.csv', index=0)
submitData_lgbm.to_csv('submission_lgbm.csv', index=0)
submitData_glm.to_csv('submission_glm.csv', index=0)

In [14]:
"""
        private    public
svm       0.515     0.510
rf        0.670     0.713
lgbm      0.715     0.73
glm       0.474     0.488
"""

'\n        private    public\nsvm       0.515     0.510\nrf        0.670     0.713\nglm       0.474     0.488\n'